# Query atlas.brabant.nl

See https://atlas.brabant.nl/arcgis/rest/services for list of available datasources.

Arcgis Python API: https://developers.arcgis.com/python/

## Libraries

In [ ]:
import requests
import pandas as pd
from arcgis.mapping import MapImageLayer
from arcgis.features import SpatialDataFrame
import geopandas as gpd

## Retrieve data and store as shapefiles

In [ ]:
def get_arcgis_data(base_url="https://atlas.brabant.nl/arcgis/rest/services",
                    service_name="stortplaatsen",
                    layer_numbers=None,
                    output_dir=None,
                    verbose=False):
    """
    Retrieves data from ArcGIS MapServer, selects layer and converts to 
    spatial dataframe. Optionally saves results as shapefiles.
    
    Arguments:
        base_url:      ArcGIS base url (usually ending with 'services')
        service_name:  Name of arcgis service provided (usually visible when accessing base_url in browser)
        layer_numbers: List with layer numbers to select. If None, all layers will be selected.
        output_dir:    Optional directory for results. Output shapefiles stored in subdir with name of service_name.
        verbose:       If True, show intermediate steps.
        
    Returns:
        Dict with layer names as keys and spatial dataframes with results as values.
    """
    
    mapserver_url = "%s/%s/MapServer" % (base_url, service_name)

    if verbose:
        print("Querying url: %s" % mapserver_url)
    map_image_layer = MapImageLayer(mapserver_url)
    
    if verbose:
        print("Available layers:")
        for layer in map_image_layer.layers:
            print("    %s" % layer.properties['name'])
    
    
    if layer_numbers == None:
        # TODO
        layer_numbers = []
    
    
    results = {}
    for layer_number in layer_numbers:
        
        # Create spatial dataframe from layer
        layer = map_image_layer.layers[layer_number]
        sdf = pd.DataFrame.spatial.from_layer(layer)
        
        # Save spatial dataframe to shapefile
        layer_name = layer.properties['name']
        if output_dir is not None:
            output_file = "%s/%s/%s.shp" % (output_dir, service_name, layer_name)
            if verbose:
                print("Saving layer %s to shapefile: %s" % (layer_number, output_file))
            sdf.spatial.to_featureclass(location=output_file)
        
        # Append spatial dataframe to dictionar
        results.update({layer_name: sdf})
    
    return results

In [ ]:
arcgis_results = get_arcgis_data(base_url='https://atlas.brabant.nl/arcgis/rest/services',
                                 service_name='stortplaatsen',
                                 layer_numbers=[3,4],
                                 output_dir='../results',
                                 verbose=True)

## Visualize using geopandas

In [ ]:
gdf_stortplaatsen = gpd.read_file('../results/stortplaatsen/4 Voormalige stortplaatsen vlak.shp')
gdf_stortplaatsen.head(3)

In [ ]:
gdf_stortplaatsen.plot()

# CURRENT: get_services

In [ ]:
def get_arcgis_services(base_url = "https://atlas.brabant.nl/ArcGIS/rest/services"):
    
    url = "%s?f=pjson" % base_url
    
    # Retrieve service list in json, convert to dataframe
    r = requests.get(url)
    results = r.json()
    df_services = pd.DataFrame(results['services'])
    
    return df_services

In [ ]:
df_services = get_arcgis_services()
df_services.head()